<a href="https://colab.research.google.com/github/wjunior15/loan_prediction/blob/main/Loan_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loan Prediction Hackathon
##Problema de Machine Learning utilizando regressão logistica binária
##Principais ferramentas: Python, Pandas, Numpy, Scikit Learn
###Objetivo: Prever se um empréstimo será aprovado ou não com base nos dados do solicitante do empréstimo.
###Site do hackathon: [DataHack](https://datahack.analyticsvidhya.com/contest/practice-problem-loan-prediction-iii/#LeaderBoard)
###Meu Score: 0.784722222222222
###Última modificação no código: 23/12/2020 

In [1]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_csv('https://raw.githubusercontent.com/shrikant-temburwar/Loan-Prediction-Dataset/master/test.csv')
train = pd.read_csv('https://github.com/shrikant-temburwar/Loan-Prediction-Dataset/raw/master/train.csv')
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


##Quantidade de valores nulos em cada coluna

In [3]:
lista_nan = []

for i in train.columns:
  valor = len(train) - (train[i].value_counts().sum())
  linha = {'coluna':i, 'valor':valor}
  lista_nan.append(linha)

##Função genérica que transforma qualquer coluna não numérica em números
OBS: os resultados não fazem sentido lógico, apenas a ordem da função value_counts()

In [4]:
def transforma_numero(dados, coluna):
  lista_numeros = []
  for i in range(len(dados)):
    for j in range(len(dados[coluna].value_counts())):
      if dados[coluna].iloc[i] == dados[coluna].value_counts().index[j]:
        lista_numeros.append(j)
    if pd.isnull(dados[coluna].iloc[i]):
      lista_numeros.append(np.nan)
  return lista_numeros

##Função que automatiza processo de transformação

In [5]:
lista_colunas_transforma = ['Gender', 'Married', 'Education', 'Dependents', 'Self_Employed', 'Property_Area', 'Loan_Status']

def automatiza_transforma(dados, colunas):
  for i in colunas:
    dados[i+'_number'] = transforma_numero(dados, i)
  
  return dados

In [6]:
train = automatiza_transforma(train, lista_colunas_transforma)
train.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Gender_number,Married_number,Education_number,Dependents_number,Self_Employed_number,Property_Area_number,Loan_Status_number
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,0.0,1.0,0,0.0,0.0,1,0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,0.0,0.0,0,1.0,0.0,2,1
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,0.0,0.0,0,0.0,1.0,1,0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,0.0,0.0,1,0.0,0.0,1,0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,0.0,1.0,0,0.0,0.0,1,0
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,0.0,0.0,0,2.0,1.0,1,0
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y,0.0,0.0,1,0.0,0.0,1,0
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N,0.0,0.0,0,3.0,0.0,0,1
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y,0.0,0.0,0,2.0,0.0,1,0
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N,0.0,0.0,0,1.0,0.0,0,1


##Função que inverte os valores Y->0 e N->1 para Y->1 e N->0 para melhorar a visualização

In [7]:
def inverte_valores(dados, coluna):
  for i in range(len(dados)):
    if dados[coluna].iloc[i] == 1:
      dados[coluna].iloc[i] = 0
    elif dados[coluna].iloc[i] == 0:
      dados[coluna].iloc[i] = 1
  return dados

In [8]:
def automatiza_inverte(dados, lista_inverte):

  for i in lista_inverte:
    dados = inverte_valores(dados, i)

  return dados

In [9]:
lista_inverte_train = ['Married_number', 'Education_number', 'Loan_Status_number']
train = automatiza_inverte(train, lista_inverte_train) 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


##Selecionando apenas as colunas desejadas para a analise

In [10]:
train = train[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Gender_number', 'Married_number', 'Dependents_number', 'Education_number', 'Self_Employed_number', 'Property_Area_number', 'Loan_Status_number']]

##Funções que analisam a média de uma variável levando em consideração o valor de uma ou duas outras variáveis

In [11]:
def analisa_media_uma(dados, coluna1, coluna2):
  ##coluna 1 - search mean
  ##coluna 2 - analysed values
  values_2 = dados[coluna2].value_counts().index
  lista_medias = []
  for i in values_2:
      valor = dados.query("{} == @i".format(coluna2))[coluna1].mean()
      if not(pd.isnull(valor)): valor = np.around(valor, 2)
      tamanho = len(dados.query("{} == @i".format(coluna2)))
      lista_medias.append({'valor': valor, 'tamanho': tamanho, 'c2':i})
  return lista_medias

In [12]:
def analisa_media_duas(dados, coluna1, coluna2, coluna3):
  ##coluna 1 - search mean
  ##coluna 2 e 3- analysed values
  values_2 = dados[coluna2].value_counts().index
  values_3 = dados[coluna3].value_counts().index
  lista_medias = []
  for i in values_2:
    for j in values_3:
      valor = dados.query("{} == @i and {} == @j".format(coluna2, coluna3))[coluna1].mean()
      if not(pd.isnull(valor)): valor = np.around(valor, 2)
      tamanho = len(dados.query("{} == @i and {} == @j".format(coluna2, coluna3)))
      lista_medias.append({'valor': valor, 'tamanho': tamanho, 'c2':i, 'c3':j})
  return lista_medias

In [13]:
analisa_media_uma(train, 'LoanAmount', 'Credit_History')

[{'c2': 1.0, 'tamanho': 475, 'valor': 144.79},
 {'c2': 0.0, 'tamanho': 89, 'valor': 146.72}]

In [14]:
analisa_media_duas(train, 'LoanAmount', 'Loan_Amount_Term', 'Married_number')

[{'c2': 360.0, 'c3': 1.0, 'tamanho': 328, 'valor': 157.74},
 {'c2': 360.0, 'c3': 0.0, 'tamanho': 183, 'valor': 128.32},
 {'c2': 180.0, 'c3': 1.0, 'tamanho': 36, 'valor': 147.79},
 {'c2': 180.0, 'c3': 0.0, 'tamanho': 8, 'valor': 146.38},
 {'c2': 480.0, 'c3': 1.0, 'tamanho': 5, 'valor': 123.4},
 {'c2': 480.0, 'c3': 0.0, 'tamanho': 9, 'valor': 166.89},
 {'c2': 300.0, 'c3': 1.0, 'tamanho': 10, 'valor': 216.2},
 {'c2': 300.0, 'c3': 0.0, 'tamanho': 3, 'valor': 81.67},
 {'c2': 84.0, 'c3': 1.0, 'tamanho': 4, 'valor': 132.25},
 {'c2': 84.0, 'c3': 0.0, 'tamanho': 0, 'valor': nan},
 {'c2': 240.0, 'c3': 1.0, 'tamanho': 2, 'valor': 127.5},
 {'c2': 240.0, 'c3': 0.0, 'tamanho': 1, 'valor': 100.0},
 {'c2': 120.0, 'c3': 1.0, 'tamanho': 2, 'valor': 21.0},
 {'c2': 120.0, 'c3': 0.0, 'tamanho': 1, 'valor': 25.0},
 {'c2': 36.0, 'c3': 1.0, 'tamanho': 0, 'valor': nan},
 {'c2': 36.0, 'c3': 0.0, 'tamanho': 2, 'valor': 117.5},
 {'c2': 60.0, 'c3': 1.0, 'tamanho': 1, 'valor': 125.0},
 {'c2': 60.0, 'c3': 0.0, 'tama

##Função para preencher dados NaN nas colunas do dataset

In [15]:
lista_nan_somente = []

for i in lista_nan:
  if i['valor'] > 0:
    lista_nan_somente.append(i)

lista_nan_somente

[{'coluna': 'Gender', 'valor': 13},
 {'coluna': 'Married', 'valor': 3},
 {'coluna': 'Dependents', 'valor': 15},
 {'coluna': 'Self_Employed', 'valor': 32},
 {'coluna': 'LoanAmount', 'valor': 22},
 {'coluna': 'Loan_Amount_Term', 'valor': 14},
 {'coluna': 'Credit_History', 'valor': 50}]

##Mudança dos valores NaN para Married_number
Com valores diferentes para homens e mulheres


In [16]:
for i in range(len(train)):
  if pd.isnull(train['Married_number'].iloc[i]):
    if train['Gender_number'].iloc[i] == 0:
      train['Married_number'].iloc[i] = 1
    else:
      train['Married_number'].iloc[i] = 0

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [17]:
train['LoanAmount'].fillna(np.around(train['LoanAmount'].mean(), 0), inplace = True)
train['LoanAmount'].describe()

count    614.000000
mean     146.397394
std       84.037503
min        9.000000
25%      100.250000
50%      129.000000
75%      164.750000
max      700.000000
Name: LoanAmount, dtype: float64

In [18]:
train['Gender_number'].fillna(0, inplace = True)
train['Gender_number'].describe()

count    614.000000
mean       0.182410
std        0.386497
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: Gender_number, dtype: float64

In [19]:
train['Dependents_number'].fillna(1, inplace = True)
train['Dependents_number'].describe()

count    614.000000
mean       0.768730
std        1.003387
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        3.000000
Name: Dependents_number, dtype: float64

In [20]:
train['Self_Employed_number'].fillna(0, inplace = True)
train['Self_Employed_number'].describe()

count    614.000000
mean       0.133550
std        0.340446
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: Self_Employed_number, dtype: float64

##Os valores vazio foram preenchidos de acordo com a porcentagem dos valores em relação aos dados validos


In [21]:
for i in train['Credit_History'].value_counts():
  print(np.around((i / train['Credit_History'].count()) * 100, 0))

84.0
16.0


In [22]:
cont = 0

for i in range(len(train['Credit_History'])):
  if pd.isnull(train['Credit_History'].iloc[i]):
    if cont < 42 :
      train['Credit_History'].iloc[i] = 1
      cont = cont + 1
    else:
      train['Credit_History'].iloc[i] = 0

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [23]:
train['Loan_Amount_Term'].fillna(np.around(train['Loan_Amount_Term'].mean(), 0), inplace = True)

##Implementando algoritmo de ML para prever se os empréstimos serão ou não aprovados
###O dataset train será dividido em dois sendo 2/3 e 1/3 para que possamos analisar a acurracia

In [24]:
X = train.iloc[:, 0:11]
Y = train.iloc[:, 11]

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 1/3, random_state = 0)

###Treinando o algoritmo com os dados de treino

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

###Fazendo a primeira predição com dados supervisionados

In [27]:
y_pred_1 = logreg.predict(X_test)
y_pred_1

array([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1])

In [28]:
print("The accuracy is {}".format(metrics.accuracy_score(y_pred_1, y_test)))

The accuracy is 0.824390243902439


##Tratar variavéis categóricas dos dados de teste

In [29]:
lista_nan = []

for i in test.columns:
  valor = len(test) - (test[i].value_counts().sum())
  linha = {'coluna':i, 'valor':valor}
  lista_nan.append(linha)

In [30]:
lista_colunas_transforma_test = ['Gender', 'Married', 'Education', 'Dependents', 'Self_Employed', 'Property_Area']
lista_inverte_test = ['Married_number', 'Education_number']

In [31]:
test = automatiza_transforma(test, lista_colunas_transforma_test)
test = automatiza_inverte(test, lista_inverte_test)
test = test[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Gender_number', 'Married_number', 'Dependents_number', 'Education_number', 'Self_Employed_number', 'Property_Area_number']]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


##Tratar dados nulos conjunto de test

In [32]:
def completa_credit_history(dados, n):
  cont = 0

  for i in range(len(dados['Credit_History'])):
    if pd.isnull(dados['Credit_History'].iloc[i]):
      if cont < n :
        dados['Credit_History'].iloc[i] = 1
        cont = cont + 1
      else:
        dados['Credit_History'].iloc[i] = 0
  
  return dados

In [33]:
test['Gender_number'].fillna(0, inplace = True )
test['Dependents_number'].fillna(1, inplace = True )
test['Self_Employed_number'].fillna(0, inplace = True )
test['LoanAmount'].fillna(np.around(test['LoanAmount'].mean(), 0), inplace = True )
test['Loan_Amount_Term'].fillna(np.around(test['Loan_Amount_Term'].mean(), 0), inplace = True )
test = completa_credit_history(test, 20)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


#Analise final dos dados de teste 

In [34]:
y_test_final = logreg.predict(test)

In [35]:
y_test_final

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

##Criando arquivo conforme moldes para envio no site da Analytics Vidhya

In [36]:
test_to_send = pd.read_csv('https://raw.githubusercontent.com/shrikant-temburwar/Loan-Prediction-Dataset/master/test.csv')

In [37]:
test_to_send['Loan_Status'] = y_test_final

In [38]:
def retransforma_status(dados):
  for i in range(len(dados)):
    if dados['Loan_Status'].iloc[i] == 1:
      dados['Loan_Status'].iloc[i] = 'Y'
    else:
      dados['Loan_Status'].iloc[i] = 'N'

  return dados

In [39]:
test_to_send = retransforma_status(test_to_send)
test_to_send.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban,Y
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban,Y
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban,Y
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban,Y
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban,Y


##Exportando arquivo CSV, sem coluna de index

In [41]:
test_to_send[['Loan_ID', 'Loan_Status']].to_csv('dados_prediction.csv', index=False)